# Preparativos

In [29]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [30]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

urlTrain  = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures_train.csv'
urlFull = 'https://raw.githubusercontent.com/CyberJuan55/Proyecto-IA/master/DataSet/SpotifyFeatures.csv'
# el train que nos dieron los profes
#df_train_original= pd.read_csv('/content/SpotifyFeatures_train.csv')
df_train_original = pd.read_csv(urlTrain)
df_train = df_train_original.copy()
# el dataset nuestro
df_full_original = pd.read_csv(urlFull)
df_full = df_full_original.copy()


# Exploracion de datos

In [31]:
df_train.dtypes

Unnamed: 0            int64
genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

# Limpieza de datos

In [32]:
df_train['genre'].value_counts()

Comedy              7771
Soundtrack          7758
Jazz                7634
Indie               7603
Children’s Music    7554
Pop                 7532
Electronic          7500
Rock                7455
Hip-Hop             7422
Folk                7406
Classical           7395
Rap                 7367
Alternative         7342
Soul                7280
R&B                 7242
World               7211
Blues               7187
Ska                 7154
Reggaeton           7139
Anime               7131
Reggae              7019
Dance               6950
Country             6861
Opera               6612
Movie               6261
Children's Music    4298
A Capella             96
Name: genre, dtype: int64

Vemos que la columna Children esta repetida

### Busqueda de datos repetidos

# Variables Categoricas

In [33]:
# Variable categoricas
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['genre', 'artist_name', 'track_name', 'track_id', 'key', 'mode', 'time_signature']


In [34]:
list_of_timeSignature = df_train['time_signature'].unique()
print(list_of_timeSignature)

['4/4' '3/4' '5/4' '1/4' '0/4']


In [35]:
df_train['time_signature'].value_counts()

4/4    160645
3/4     19247
5/4      4198
1/4      2083
0/4         7
Name: time_signature, dtype: int64

Decidimos no utilizar Time_signature como feature por varios motivos


*   En musica no existe compaces de 1/4 ni de 0/4 entre ambos suman 1656 valores lo que nos parecio un numero considerable
*   En una cancion puede haber multiples cambios de compaces, en la musica progresiva es algo muy comun

*   La mayoria de la musica se encuentra en 4/4 podemos ver que hay 156851 valores que considerablemente mayor a lo suma de todas las otras categorias esto introduceria un gran sesgo en nuestro modelo 










In [36]:
# variables categoricas con baja cardinalidad
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 14 and 
                    df_train[cname].dtype == "object"]
print(categorical_cols)

['key', 'mode', 'time_signature']


Quedan descartadas las variables categoricas: 

*   Time_signature por los motivos comentados anteriormente
*   track_id: no hay ninguna razon para utilizarla
*   el nombre del artista: son demasiadas variables de momento las sacamos pero es cierto que es un feature interesente 
*   track_name: todos sus valores son distintos 







In [37]:
list_of_modes = df_train['mode'].unique()
print(list_of_modes)

['Major' 'Minor']


In [38]:
list_of_keys = df_train['key'].unique()
print(list_of_keys)


['B' 'A#' 'C#' 'C' 'F#' 'E' 'A' 'D' 'G' 'F' 'G#' 'D#']


### Preprocesado de datos

In [39]:
# Funcion para el preprocesamiento de datos


def preprocesado(df):

    if df.duplicated().sum() == 0:
        print('no hay datos duplicados')
    else:
        df.drop_duplicates


    if 'Unnamed: 0' in df.columns:
        df = df.drop(['Unnamed: 0'], axis=1)
    else:
        print('no esta')  

    df = df.drop(['time_signature','track_id','artist_name','track_name'], axis=1)
    indexNamesComedy = df[ df['genre'] == 'Comedy' ].index
    indexNamesChildren = df[ df['genre'] == "Children's Music" ].index
    df.drop(indexNamesComedy , inplace=True)
    df.drop(indexNamesChildren , inplace=True)
    
    cols_with_missing = [col for col in df.columns if df[col].isnull().any()] 
    faltanDatos = True
    if len(cols_with_missing)==0:
        faltanDatos= False
        print('no faltan valores') 
    if faltanDatos == True:
        print(cols_with_missing)
        df.dropna(axis=0, inplace=True)

    X = df.drop(columns=['genre'])
    y = df['genre']
    

    return X,y 


In [40]:
X,y = preprocesado(df_train)

no hay datos duplicados
no faltan valores


In [41]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [42]:

X_train_full.select_dtypes('object')
binary_features = X_train_full['mode']
nominal_features = X_train_full['key']


In [43]:
# Transformacion de datos 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


binary_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories='auto'))
])
nominal_transformer = Pipeline(steps=[
    ('nominal', OneHotEncoder())
])


In [44]:
preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, binary_features),
    ('nominal', nominal_transformer, nominal_features)
], sparse_threshold=0)

In [45]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

In [46]:
model.fit(X_train_full, y_train)

ValueError: ignored

# Un poco de ingenieria de datos